In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/data-set/train.csv")

In [ ]:
y=df["CustomerAttrition"]


In [ ]:
x=df[['sex', 'Aged', 'Married', 'TotalDependents', 'ServiceSpan',
       'MobileService', '4GService', 'CyberProtection', 'HardwareSupport',
       'TechnicalAssistance', 'FilmSubscription', 'SettlementProcess',
       'QuarterlyPayment', 'GrandPayment']]

In [ ]:
missing_values_count = df.isnull().sum()

In [ ]:
missing_values_count

In [ ]:
cols_with_missing = [col for col in df.columns
                     if df[col].isnull().any()]

In [ ]:
from sklearn.impute import SimpleImputer
for col in cols_with_missing:
    df[col + '_was_missing'] = df[col].isnull()

In [ ]:
#df=df.fillna(0)
mean_value=df['GrandPayment'].mean()
df['GrandPayment'].fillna(value=mean_value, inplace=True)
#df['GrandPayment']=df['GrandPayment'].fillna(value=mean_value, inplace=True)

In [ ]:
(df.GrandPayment==mean_value).sum()

In [ ]:
x=df[['sex', 'Aged', 'Married', 'TotalDependents', 'ServiceSpan',
       'MobileService', '4GService', 'CyberProtection', 'HardwareSupport',
       'TechnicalAssistance', 'FilmSubscription', 'SettlementProcess',
       'QuarterlyPayment', 'GrandPayment']]#,'GrandPayment_was_missing']]

In [ ]:
y=df[ 'CustomerAttrition']

In [ ]:
x.isnull().sum()

In [ ]:
x.corr()

In [ ]:
s = (x.dtypes == 'object')  # || x.dtypes=='bool')
object_cols = list(s[s].index)

#object_cols.append('GrandPayment_was_missing')
print("Categorical variables:")
print(object_cols)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
y2=y

In [ ]:
y= label_encoder.fit_transform(y)

In [ ]:
from xgboost import XGBClassifier,XGBRegressor

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(x[object_cols]))
#OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = x.index
#OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = x.drop(object_cols, axis=1)
#num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X = pd.concat([num_X_train, OH_cols_train], axis=1)
#OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)



In [ ]:
scores =   cross_val_score(tesla_model ,OH_X ,y,
                             cv=5,
                            scoring='accuracy')

print("MAE scores:\n", scores)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
lsvc = LinearSVC(C=0.05, penalty="l1", dual=False)

In [ ]:
mean = OH_X.mean(axis=0)
OH_X -= mean
std = OH_X.std(axis=0)
OH_X /= std

In [ ]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(OH_X, y, stratify=y, random_state=42, test_size = 0.2)

In [ ]:
from keras.utils.np_utils import to_categorical

Y_train = to_categorical(y_train, num_classes=None)
Y_test = to_categorical(y_test, num_classes=None)
print (Y_train.shape)
print (Y_train[:10])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout
from keras import regularizers

# define a function to build the keras model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(32, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(28,)))
    model.add(Dense(32, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(8, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.add(Dense(8, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(16, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.add(Dense(16, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation='softmax'))
    
    # compile model
    adam = Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

model = create_model()

print(model.summary())

In [ ]:
X_train.shape

In [ ]:
 Y_train.shape

In [ ]:
X_test.shape

In [ ]:
Y_test.shape

In [ ]:
history=model.fit(X_train, Y_train, validation_data=(X_test, Y_test),epochs=100, batch_size=10)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Model accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# Model Losss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
tesla_model = RandomForestClassifier(max_leaf_nodes=40,random_state = 5)
tesla_model1=LinearSVC(C=0.05, penalty="l1", dual=False)
X_train, X_test, y_train, y_test = train_test_split(normalized_X, y, test_size=0.2, random_state=42)
tesla_model1.fit(X_train, y_train)
mae = mean_absolute_error(y_test,tesla_model1.predict(X_test))
mae

0.20425867507886436

In [ ]:
tesla_model = RandomForestClassifier(max_leaf_nodes=40,random_state = 5)
tesla_model2=LinearSVC(C=0.07, penalty="l1", dual=False)
X_train, X_test, y_train, y_test = train_test_split(normalized_X, y, test_size=0.2, random_state=42)
tesla_model2.fit(X_train, y_train)
mae = mean_absolute_error(y_test,tesla_model2.predict(X_test))
mae

In [ ]:
tesla_model = DecisionTreeClassifier(max_leaf_nodes=50,random_state = 1)
X_train, X_test, y_train, y_test = train_test_split(OH_X, y, test_size=0.2, random_state=42)
tesla_model.fit(X_train, y_train)
mae = mean_absolute_error(y_test,tesla_model.predict(X_test))

In [ ]:
#model=XGBClassifier(learning_rate =0.008,n_estimators=1200,max_depth=6)
model2=XGBClassifier()

In [ ]:
from sklearn import svm
clf = svm.SVC()

In [ ]:
from sklearn import preprocessing
normalized_X = preprocessing.scale(OH_X)

In [ ]:
from sklearn import datasets, linear_model
lasso = linear_model.Lasso()

In [ ]:
from sklearn.svm import SVR
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)

In [ ]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores =   cross_val_score(clf , normalized_X  ,y,
                             cv=5,
                            scoring='accuracy')

print("MAE scores:\n", scores)

In [ ]:
print("Average MAE score (across experiments):")
print(scores.mean())

Average MAE score (across experiments):
0.21000375959505924

Average MAE score (across experiments) svm:
0.2650678843439009

0.24282315711372648

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
  
# Creating the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
# Instantiating logistic regression classifier
from sklearn import svm
clf = svm.SVC()
  
# Instantiating the GridSearchCV object
logreg_cv = GridSearchCV(clf , param_grid, cv = 5)
  
logreg_cv.fit(normalized_X, y)
  
# Print the tuned parameters and score
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

In [ ]:
clf3=svm.SVC(C= 1000, gamma= 10, kernel= 'rbf')

In [ ]:
scores =   cross_val_score(svm_poly_reg , normalized_X  ,y,
                             cv=5)

print("MAE scores:\n", scores)
print("Average MAE score (across experiments):")
print(scores.mean())

In [ ]:
from sklearn.model_selection import train_test_split
OH_X_train, OH_X_valid, y_train, y_valid=train_test_split(OH_X,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBClassifier,XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import svm #Perforing grid search
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    #model = GridSearchCV(estimator = XGBClassifier( learning_rate=0.01, n_estimators=1000, max_depth=4,cv=5) ,param_grid = param_test1, scoring='roc_auc',n_jobs=4, cv=5)
    #model =XGBRegressor( learning_rate =0.008,n_estimators=1200,max_depth=6)
    clf = svm_poly_reg
    clf.fit(X_train, y_train)
    #model.fit(X_train, X_train)
    preds = clf.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

In [ ]:
svm_poly_reg.fit(normalized_X  ,y)

OH_X_train, OH_X_valid, y_train, y_valid

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier

def get_mae(max_leaf_nodes,OH_X_train, OH_X_valid, y_train, y_valid):
    model = DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
    model.fit(OH_X_train, y_train)
    preds_val = model.predict(OH_X_valid)
    mae = mean_absolute_error(y_valid, preds_val)
    return(mae)

In [ ]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes,OH_X_train, OH_X_valid, y_train, y_valid)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

In [ ]:
OH_X_train.shape


In [ ]:
mae

In [ ]:
df2=pd.read_csv("../input/data-set/test.csv")

In [ ]:
df2.columns

In [ ]:
k=df2.iloc[0:,1:15]

In [ ]:
k.columns

In [ ]:
s = (k.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(k[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_test.index = k.index


# Remove categorical columns (will replace with one-hot encoding)
num_test = k.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_y = pd.concat([num_test, OH_cols_test], axis=1)

In [ ]:
c=OH_y.isnull().sum()

In [ ]:
mean_value=OH_y['GrandPayment'].mean()
OH_y['GrandPayment'].fillna(value=mean_value, inplace=True)

In [ ]:
ml=tesla_model1.predict(OH_y)

In [ ]:
ml

In [ ]:
ml2=np.round(ml)

In [ ]:
df3=df2['ID']

In [ ]:
ml2 = pd.Series(ml2)

In [ ]:
df3 = pd.DataFrame(df3)

In [ ]:
ml2 = ml2.rename('CustomerAttrition')

In [ ]:
df4 = df3.join(ml2)

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe
df = pd.DataFrame(np.random.randn(50, 4), columns=list('ABCD'))

# create a link to download the dataframe
create_download_link(df4)

# ↓ ↓ ↓  Yay, download link! ↓ ↓ ↓ 